# Validate Minian outputs - 2nd version

- Load
    - spatial map (A) -> maybe to associate with a projection of neuron fluorescence
    - temporal activity of detected neurons

- Validate units

In [ ]:
import numpy as np
import csv
from pathlib import Path
import xarray as xr
import pandas as pd
from pandas import concat
import matplotlib.pyplot as plt
import itertools as itt
import os
import sys
import json

import holoviews as hv
from dask.distributed import Client, LocalCluster
from holoviews.operation.datashader import datashade, regrid
from holoviews.util import Dynamic
from holoviews import opts
from holoviews.operation.datashader import shade
hv.extension('bokeh', 'matplotlib')
from IPython.core.display import display

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)


Import the minian files

In [ ]:
folderGen = Path('.').absolute()
experimenter = 'Gaelle'
recording = 'Baseline_recording'#Baseline_recording
mouse_name = 'RedLinesOK'
date = 'session3'
time = 'Rec1'
device = 'V4_Miniscope'
minianversion = 'minian'
device2 = 'V4_Miniscope'

folderMouse = Path(f'{folderGen}/{experimenter}/{recording}/{mouse_name}/{date}/{device}/{minianversion}/')
minian_ds = open_minian(folderMouse)
folder_base = Path(f'{folderGen}/{experimenter}/{recording}/{mouse_name}/')

A = minian_ds['A']
C = minian_ds['C']
S = minian_ds['S']

B = A['unit_id']
series = B.to_series()
D = series.count()

idloc = A.idxmax("unit_id")
Hmax = A.idxmax("height")
Hmax2 = Hmax.max("width")

Wmax = A.idxmax("width")
Wmax2 = Wmax.max("height")
coord1 = Wmax2.to_series()
coord2 = Hmax2.to_series()

a = pd.concat([coord1,coord2], axis=1)
unit = len(a)
print(unit)

AllUnitsFile = folderMouse / f'AllunitsFile.json'
all_units = a.index.tolist()

with open(AllUnitsFile, 'w') as f:
    json.dump(all_units, f, indent=2) 

In [ ]:
with open(AllUnitsFile, 'w') as f:
    json.dump(all_units, f, indent=2) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5)) 
ax1.imshow(A[0,:])
ax2.plot(C[0,:])
ax2.annotate(f'Unit_id {0}', (12, 3), fontsize=12)

Plot the spatial map for all cells + interactive Ca2+ trace

In [ ]:
NeuronToPlot = range(unit)

output_size = 150
hv.output(size=int(output_size))
opts = dict(
    plot=dict(colorbar=True, invert_yaxis=True),
    style=dict(cmap="Viridis"),
)
image = hv.Image(
    A.max("unit_id").compute().astype(np.float32).rename("A"),
    kdims=["width", "height"],
).opts(**opts)
labels = hv.Labels([(a.iloc[i,0], a.iloc[i,1], a.index[i]) for i in NeuronToPlot])
plot_unit = hv.HoloMap({i: hv.Curve(((C[i,:])), group='Group', label=f'{a.index[i]}') for i in NeuronToPlot}, 'Value').opts(ylim=(-0.5, 20))
labels2 = hv.Labels((150, 150, a.index[1]))
layout = image * labels + plot_unit * labels2
layout

Remove dropped units

In [ ]:
# fill in unit_to_drop with cell_id of cells to drop
unit_to_drop = [2, 30]
copyB = list(B.copy())
for i in range(len(unit_to_drop)):
    elem = unit_to_drop[i]
    copyB.remove(elem)
unit_to_keep = copyB
unit_to_keepL = [e for e in all_units if e not in unit_to_drop]

A_upd = A.loc[unit_to_keep,:,:]
C_upd = C.loc[unit_to_keep,:]
S_upd = S.loc[unit_to_keep,:]

TodropFile = folderMouse / f'TodropFile.json'


with open(TodropFile, 'w') as f:
    json.dump(unit_to_drop, f, indent=2) 

listtokeep = [9, 1, 15, 19, 25, 27, 28, 29, 30, 34, 41, 48, 53, 60]
unit_to_keeppd = pd.Series(unit_to_keepL)
filenameOut = folderMouse / f'UnitToKeep.xlsx'
writer = pd.ExcelWriter(filenameOut)
unit_to_keeppd.to_excel(writer)
writer.save()

In [ ]:
listtokeep = [9, 15, 25, 27, 28, 29, 30, 34, 41, 48, 53, 60]
unit_to_keeppd = pd.Series(listtokeep)
filenameOut = folderMouse / f'UnitToKeep.xlsx'
writer = pd.ExcelWriter(filenameOut)
unit_to_keeppd.to_excel(writer)
writer.save()

In [ ]:
MapSessionsPath = folder_base / f'mappings.pkl'

MapSessions = pd.read_pickle(MapSessionsPath)
MapSessions